# Simulador de temperatura y humedad


In [1]:
import os
from datetime import time
import json
import pandas as pd
import numpy as np
import pickle

## Variables globales

In [2]:
FIC_CON_RAW="00_burgos_concatenated_output.json"
FIC_SALIDA="01_Temp_Humedad.pkl"
RUTA_DATOS = r"C:\Users\jaume\Documents\Proyecto\datos"

## Descarga de datos de temperatura y humedad


He descargado los datos historicos de Belorado (Burgos) pues es donde vamos a tomar las temperaturas y humedades de donde vamos a realizar el modelado.

He solicitado el token y como son datos iniciales que he querido que fueran reales y tengo que realizar 3 peticiones ya solo podemos descargar 6 meses de datos por peticion. Pero como son pocos lo he hecho usando un método manual descrito [aquí](https://www.youtube.com/watch?v=wGNYqLOq4fE) y [aquí](https://www.youtube.com/watch?v=gntHivOmT_U)

He usado la siguiente estación meteorológica: Belorado (Burgos): 9111

Ficheros originales: datos_burgos[123].json


## Concatenamos los 3 ficheros

In [3]:
os.chdir(RUTA_DATOS)

# Lista de archivos JSON
json_files = [ "datos_burgos.json", "datos_burgos2.json", "datos_burgos3.json" ]

# Inicializar una lista vacía para los datos concatenados
concatenated_data = []

# Leer y concatenar los datos
for file in json_files:
    with open(file, "r") as f:
        data = json.load(f)  # Cargar el JSON como lista
        concatenated_data.extend(data)  # Extender la lista con los datos

# Guardar el resultado en un nuevo archivo JSON
with open(FIC_CON_RAW, "w") as output_file:
    json.dump(concatenated_data, output_file, indent=4)

print(f"Archivos concatenados exitosamente en '{FIC_CON_RAW}'")

Archivos concatenados exitosamente en '00_burgos_concatenated_output.json'


## Pre-Procesado de datos

In [4]:
columnas = [ "fecha", "tmin", "tmed", "tmax", "horatmin", "horatmax", "hrMin", "hrMedia", "hrMax", "horaHrMin", "horaHrMax" ] 

df_raw = pd.read_json(FIC_CON_RAW)
df_raw.describe()
df_raw_columnas = df_raw[columnas].copy()

In [5]:
df_raw_columnas

,fecha,tmin,tmed,tmax,horatmin,horatmax,hrMin,hrMedia,hrMax,horaHrMin,horaHrMax
0,2023-01-01,"6,9","12,0","17,2",07:10,Varias,28.0,45.0,73.0,18:00,07:00
1,2023-01-02,"3,6","7,7","11,8",23:59,NaN,44.0,76.0,93.0,00:00,23:59
2,2023-01-03,"0,1","4,9","9,7",07:00,14:10,44.0,67.0,96.0,14:40,Varias
3,2023-01-04,"-1,0","5,4","11,9",07:30,14:20,44.0,57.0,88.0,12:00,23:59
4,2023-01-05,"1,1","6,6","12,0",07:20,14:40,56.0,78.0,96.0,15:00,Varias
...,...,...,...,...,...,...,...,...,...,...,...
542,2024-06-27,"15,5","22,4","29,4",05:00,14:50,41.0,66.0,97.0,14:50,Varias
543,2024-06-28,"15,1","21,6","28,1",23:59,15:00,37.0,64.0,98.0,14:30,Varias
544,2024-06-29,"13,7","16,7","19,7",Varias,14:50,56.0,80.0,99.0,15:40,23:30
545,2024-06-30,"12,5","17,2","22,0",23:59,14:00,46.0,67.0,98.0,16:30,Varias


### Limpieza de datos

In [6]:
df_limpio_tmp = df_raw_columnas.copy()
df_limpio_tmp.isna().sum()
#df_limpio_tmp.isna()

fecha         0
tmin          7
tmed          8
tmax          8
horatmin      7
horatmax     11
hrMin         8
hrMedia       3
hrMax         8
horaHrMin     8
horaHrMax     8
dtype: int64

#### Columna "fecha"

Primero verificamos que la columna fecha:

* Contengo una fila por fecha, sin repeticiones
* Que no haya entradas erroneas
* Que no falte ninguna fecha en el rango

In [7]:
# Verificar fechas no válidas (si el formato es inconsistente)
df_invalid = df_limpio_tmp[~pd.to_datetime(df_limpio_tmp["fecha"], errors="coerce").notna()]
print("Fechas no válidas:")
print(df_invalid)

# Verificar fechas duplicadas
df_duplicated = df_limpio_tmp[df_limpio_tmp["fecha"].duplicated(keep=False)]
print("Fechas duplicadas:")
print(df_duplicated)

# Verificar fechas faltantes
rango_fechas = pd.date_range(df_limpio_tmp["fecha"].min(), df_limpio_tmp["fecha"].max())
fechas_faltantes = rango_fechas.difference(pd.to_datetime(df_limpio_tmp["fecha"]))
print("Fechas faltantes:")
print(fechas_faltantes)


Fechas no válidas:
Empty DataFrame
Columns: [fecha, tmin, tmed, tmax, horatmin, horatmax, hrMin, hrMedia, hrMax, horaHrMin, horaHrMax]
Index: []
Fechas duplicadas:
Empty DataFrame
Columns: [fecha, tmin, tmed, tmax, horatmin, horatmax, hrMin, hrMedia, hrMax, horaHrMin, horaHrMax]
Index: []
Fechas faltantes:
DatetimeIndex(['2023-11-02'], dtype='datetime64[ns]', freq='D')


Como vemos parece que hay algunas fechas que no están, las insertamos con el siguiente código:

In [8]:
df = pd.DataFrame(df_limpio_tmp)

# Asegúrate de que la columna fecha es de tipo datetime
df["fecha"] = pd.to_datetime(df["fecha"])

# Generar el rango completo de fechas
fecha_min = df["fecha"].min()
fecha_max = df["fecha"].max()
rango_completo = pd.date_range(start=fecha_min, end=fecha_max)

# Encontrar las fechas faltantes
fechas_faltantes = rango_completo.difference(df["fecha"])

# Crear filas con las fechas faltantes y NaN en las demás columnas
filas_faltantes = pd.DataFrame({
    "fecha": fechas_faltantes,
    **{col: [np.nan] * len(fechas_faltantes) for col in df.columns if col != "fecha"}
})

# Concatenar y reordenar el DataFrame por la columna fecha
df_limpio_tmp = pd.concat([df, filas_faltantes], ignore_index=True).sort_values(by="fecha").reset_index(drop=True)

# Mostrar el DataFrame actualizado
print(df_limpio_tmp.iloc[300:311])


         fecha  tmin  tmed  tmax horatmin horatmax  hrMin  hrMedia  hrMax  \
300 2023-10-28  11,1  14,3  17,5    01:00    15:30   46.0     58.0   72.0   
301 2023-10-29  11,7  15,0  18,3    15:10    13:00   52.0     68.0   92.0   
302 2023-10-30   9,1  11,2  13,3    23:59    00:10   51.0     63.0   88.0   
303 2023-10-31   7,0  10,3  13,6    20:00    13:30   50.0     64.0   79.0   
304 2023-11-01   NaN   NaN   NaN      NaN      NaN    NaN     65.0    NaN   
305 2023-11-02   NaN   NaN   NaN      NaN      NaN    NaN      NaN    NaN   
306 2023-11-03   5,5   8,6  11,7    00:20    13:20   47.0     61.0   82.0   
307 2023-11-04   NaN   NaN   NaN      NaN      NaN    NaN     72.0    NaN   
308 2023-11-05   7,6   9,8  12,1    06:30    14:30   58.0     65.0   72.0   
309 2023-11-06   4,9   7,8  10,6    23:59    14:40   49.0     61.0   75.0   
310 2023-11-07   4,2   7,0   9,9    04:10    13:40   62.0     67.0   74.0   

    horaHrMin horaHrMax  
300     15:30     07:00  
301     12:30     15:20

In [9]:
#print(df_limpio_tmp[df_limpio_tmp.isnull().any(axis=1)])

#### Limpiar columnas de mediciones

Simplemente limipio los datos numéricos para que sean float y en caso de no serlo calculo la media con el anterior y el siguiente de la fila.

In [10]:
col_numericas = ( "tmin", "tmed", "tmax", "hrMin", "hrMedia", "hrMax" )

for col in col_numericas:

    # Substituimos las "," por "." para dejar todo consistente antes de pasarlo a numérico
    df_limpio_tmp[col] = df_limpio_tmp[col].astype(str).str.replace(',', '.', regex=False)

    # Intenta convertir a float, y los que no pueda los deja como NaN
    df_limpio_tmp[col] = pd.to_numeric(df_limpio_tmp[col], errors='coerce')

    # Reemplaza los NaN usando una función lambda que busca el valor anterior y posterior
    df_limpio_tmp[col] = df_limpio_tmp[col].fillna(df_limpio_tmp[col].rolling(window=3, center=True, min_periods=1).mean())

In [11]:
print(df_limpio_tmp[df_limpio_tmp.isnull().any(axis=1)])
print(df_limpio_tmp.iloc[300:308])

         fecha   tmin   tmed   tmax horatmin horatmax  hrMin  hrMedia  hrMax  \
1   2023-01-02   3.60   7.70  11.80    23:59      NaN   44.0     76.0   93.0   
15  2023-01-16   0.95   5.30   9.65      NaN      NaN   57.5     81.0   89.5   
40  2023-02-10  -3.20   2.20   7.55      NaN      NaN   48.5     62.0   92.0   
53  2023-02-23  -0.30   2.20   4.70    22:30      NaN   89.0     98.0   99.0   
101 2023-04-12   3.20   9.00  14.90    23:59      NaN   35.0     80.0   98.0   
186 2023-07-06  13.55  20.20  26.75      NaN      NaN   46.0     67.0   91.5   
304 2023-11-01   7.00  10.30  13.60      NaN      NaN   50.0     65.0   79.0   
305 2023-11-02   5.50   8.60  11.70      NaN      NaN   47.0     63.0   82.0   
307 2023-11-04   6.55   9.20  11.90      NaN      NaN   52.5     72.0   77.0   
311 2023-11-08   5.90   8.50  11.10      NaN      NaN   56.5     66.0   84.5   
324 2023-11-21   5.35   8.45  11.55      NaN      NaN   66.5     69.0   96.5   
364 2023-12-31   5.50   7.40   9.30   Va

#### Imputación de las columnas de horas

Si los valores de la columna no existen o no son correctos los imputaremos siguiendo estas condiciones:

* horatmin, horaHrmax: Consideraré que la probabilidad que la temperatura sea más baja y la humedad más alta sea a las 5 de la mañana.
* horatmax, horaHrmin: Aquí el mediodía 12:00


In [12]:
# Convertir las columnas a formato de hora, manejando errores
for col in ['horatmin', 'horaHrMax', 'horatmax', 'horaHrMin']:
    df_limpio_tmp[col] = pd.to_datetime(df_limpio_tmp[col], format='%H:%M', errors='coerce').dt.time

# Valores predeterminados como objetos time
valores_predeterminados = {
    'horatmin': time(5, 0),
    'horaHrMax': time(5, 0),
    'horatmax': time(12, 0),
    'horaHrMin': time(12, 0)
}

# Rellenar valores NaT con valores predeterminados
df_limpio_tmp.fillna(valores_predeterminados, inplace=True)


In [13]:
#print(df_limpio_tmp.describe())
print(df_limpio_tmp.dtypes)
print(df_limpio_tmp.head())
print(df_limpio_tmp.isnull().sum())
print(df_limpio_tmp.tail())

fecha        datetime64[ns]
tmin                float64
tmed                float64
tmax                float64
horatmin             object
horatmax             object
hrMin               float64
hrMedia             float64
hrMax               float64
horaHrMin            object
horaHrMax            object
dtype: object
       fecha  tmin  tmed  tmax  horatmin  horatmax  hrMin  hrMedia  hrMax  \
0 2023-01-01   6.9  12.0  17.2  07:10:00  12:00:00   28.0     45.0   73.0   
1 2023-01-02   3.6   7.7  11.8  23:59:00  12:00:00   44.0     76.0   93.0   
2 2023-01-03   0.1   4.9   9.7  07:00:00  14:10:00   44.0     67.0   96.0   
3 2023-01-04  -1.0   5.4  11.9  07:30:00  14:20:00   44.0     57.0   88.0   
4 2023-01-05   1.1   6.6  12.0  07:20:00  14:40:00   56.0     78.0   96.0   

  horaHrMin horaHrMax  
0  18:00:00  07:00:00  
1  00:00:00  23:59:00  
2  14:40:00  05:00:00  
3  12:00:00  23:59:00  
4  15:00:00  05:00:00  
fecha        0
tmin         0
tmed         0
tmax         0
horatmin   

## Guardamos el dataframe resultante en un fichero

In [14]:
# Guardar un objeto en un fichero
with open(FIC_SALIDA, 'wb') as f:
    pickle.dump(df_limpio_tmp, f)